<h1>NetLib</h1>

In [887]:
import networkx as nx
import datetime
import pandas as pd
import numpy as np
import json
import math

import plotly
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy.spatial
import geopy.distance

from datetime import datetime

#pd.options.display.max_rows = 20
#pd.set_option('max_colwidth', 2000)
from IPython.display import Markdown, display


import matplotlib.pyplot as plt
import matplotlib.colors as cls

import importlib
import import_ipynb

import random


In [962]:
def make_random_graph(PORTS=4,SERVICES=3):
    M=np.random.choice([0, 1], size=(PORTS,SERVICES), p=[1./4, 3./4])

    G=nx.DiGraph()

    for s in range(0, M.shape[1]):
        n_color = [random.uniform(0, 1),random.uniform(0, 1),random.uniform(0, 1)]
        for p in range(0, M.shape[0]):
            G.add_node((p,s), pos=(p,s), color=n_color, demand=0)
        
    ncolor =nx.get_node_attributes(G,'color')

    for p in range(0, M.shape[0]):
        for s in range(0, M.shape[1]):
            for sprime in range(0, M.shape[1]):
                e_color = [random.uniform(0, 1),random.uniform(0, 1),random.uniform(0, 1)]
                if M[p][s]*M[p][sprime]==1 and s!=sprime:
                    dist=(s-sprime)**2
                    G.add_edge((p,s),(p,sprime), weight=dist,color=[0,0,0])



    for s in range(0, M.shape[1]):
        e_color=ncolor[(0, s)]
        for p in range(0, M.shape[0]):
            for pprime in range(0, M.shape[0]):
                if M[p][s]*M[pprime][s]==1 and p!=pprime:
                    dist=(p-pprime)**2
                    G.add_edge((p,s),(pprime,s), weight=dist, color=e_color)

    
    return G

In [898]:
def optimal_random_graph(graph, POL=(0,0), POD=(1,1)):
    
    graph.nodes[POL]['demand']=-10
    graph.nodes[POD]['demand']=10
    flowDict = nx.min_cost_flow(graph)
    dem=nx.get_node_attributes(graph,'demand')
    
    Gr=nx.DiGraph()
    

    for key, value in zip(dem.keys(),dem.values()):
        if value!=0:
            Gr.add_node(key, pos=key, color='red')
        else:
            Gr.add_node(key, pos=key, color='black')

    for key, value in zip(flowDict.keys(),flowDict.values()):
        for k,v in value.items():
            if v!=0:
                if k in list(Gr.nodes()):
                    Gr.remove_node(k)
                    Gr.add_node(k, pos=k, color='red')
                else:
                    Gr.add_node(k, pos=k, color='red')

                

    for key, value in zip(flowDict.keys(),flowDict.values()):
        for k,v in value.items():
            if v!=0:
                Gr.add_edge(key, k, weight=5, color='red')
    return Gr

In [1]:
def draw_random_graph(graph):
    
    pos=nx.get_node_attributes(graph,'pos')
    ncolor =nx.get_node_attributes (graph,'color')
    ecolor=nx.get_edge_attributes(graph,'color')
    estyle=nx.get_edge_attributes(graph, 'style')
            
    plt.figure(3,figsize=(14,7)) 
    return nx.draw(graph, pos=pos, nodelist=list(graph.nodes()), edgelist=list(graph.edges()), with_labels=True,  node_size=100, verticalalignment='top', edge_color=list(ecolor.values()), node_color=list(ncolor.values()),  width=3, arrowsize =10, style='dotted')

In [ ]:
def tracking_metro(dframe, LAT_NAME, LON_NAME, DATE_NAME=None, COLOR=None, SIZE=None, ZOOM=3):
    mapbox_access_token='pk.eyJ1IjoiYnJhZGlwbzg5IiwiYSI6ImNraDR2ZGMxcTAxbDYyenBiaWVyYmR5ZHcifQ.h004vuTCSVPy8mP95fq3Aw'
    
    LINES=['1', '2', '3', '4', '5', '6', '8', '9', '11', '12', '14', 'M3bis']
    START=['la défense - grande arche', 'nation', 'gallieni (parc de bagnolet)', 'mairie de montrouge', "place d'italie" , 'charles de gaulle-étoile', 'créteil-pointe du lac', 'pont de sèvres', 'châtelet', 'front populaire','saint-lazare', 'gambetta' ]
    
    
    if COLOR==None:
        colors = 'darkmagenta'
        showsc = False
    else:
        colors = dframe[COLOR]
        showsc = False
    
    
    if SIZE==None:
        sizes = 4
        showl = True
        MAXSIZE=1
    if type(SIZE)==int:
        sizes=SIZE
        showl = True
        MAXSIZE=1
    else:
        sizes = dframe[SIZE]
        showl = True
        MAXSIZE=dframe[SIZE].mean()/15
        

    fig = go.Figure()
    
    
    for line, end in zip(LINES, START):
        
        dframe_1 = dframe.loc[dframe.ligne==line]
        sha = dframe_1.shape[0]
    
    
        dframe_1['start_point_lat']=dframe_1.loc[dframe_1.nom_gare==end]['latitude'].item()
        dframe_1['start_point_lon']=dframe_1.loc[dframe_1.nom_gare==end]['longitude'].item()
        dframe_1['dist_from_start'] = np.sqrt((dframe_1['latitude']-dframe_1['start_point_lat'])**2+(dframe_1['longitude']-dframe_1['start_point_lon'])**2)
        dframe_1=dframe_1.sort_values(by='dist_from_start')
    

        dff=dframe_1.iloc[:1]
    
        while dff.shape[0]!=sha:
            dframe_1=dframe_1.iloc[1:]
            start_point_lat=dframe_1.iloc[0]['latitude']
            start_point_lon=dframe_1.iloc[0]['longitude']
            dframe_1['dist_from_start'] = np.sqrt((dframe_1['latitude']-start_point_lat)**2+(dframe_1['longitude']-start_point_lon)**2)
            dframe_1=dframe_1.sort_values(by='dist_from_start')
        
            dff.loc[len(dff)]=dframe_1.iloc[0]
        
        
        
        
        
        
        colors = dff.loc[dff.ligne==line]['color'].unique().item()
        lat = dff[LAT_NAME]
        lon = dff[LON_NAME]
        
        mark_1 = go.scattermapbox.Marker(size=sizes, sizemode='area',sizeref=MAXSIZE, color=colors, showscale=showsc,colorbar=dict(title='{}'.format('nom_gare'),titleside='right'))
        trace_1 = go.Scattermapbox(text=dff['nom_gare'], name = 'ligne {}'.format(line), lat=lat, lon=lon, hovertext = dff['nom_gare'], hoverinfo ='text', mode='markers+lines', marker=mark_1)
    
        fig.add_trace(trace_1)




    

    fig.update_layout(
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=48.858799,
                lon=2.347306
            ),
            pitch=0,
            zoom=ZOOM
        )
    )
    fig.update_layout(margin = dict(t=65, l=2, r=0, b=2),
                  autosize=True,
                  width=850,
                  height=700)
    
    fig.update_layout(showlegend=True)
    fig.update_layout(mapbox_style="dark")
    fig.update_layout(title = "Paris Metro Map")

    fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    itemsizing='constant'
    ))
    fig.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
    )

    return fig.show()

In [ ]:
def make_metro_graph(dframe, mode='net'):
    
    
    LINES=['1', '2', '3', '4', '5', '6', '8', '9', '11', '12', '14', 'M3bis']
    START=['la défense - grande arche', 'nation', 'gallieni (parc de bagnolet)', 'mairie de montrouge', "place d'italie" , 'charles de gaulle-étoile', 'créteil-pointe du lac', 'pont de sèvres', 'châtelet', 'front populaire','saint-lazare', 'gambetta' ]
    
    G=nx.DiGraph()
    
    
    if mode=='net':
        L=1
        
        for line, end in zip(LINES, START):
        
        
            dframe_1 = dframe.loc[dframe.ligne==line]
            sha = dframe_1.shape[0]
            color_ligne = cls.to_rgb(dframe_1.loc[dframe_1.ligne==line]['color'].unique().item())
    
    
            dframe_1['start_point_lat']=dframe_1.loc[dframe_1.nom_gare==end]['latitude'].item()
            dframe_1['start_point_lon']=dframe_1.loc[dframe_1.nom_gare==end]['longitude'].item()
            dframe_1['dist_from_start'] = np.sqrt((dframe_1['latitude']-dframe_1['start_point_lat'])**2+(dframe_1['longitude']-dframe_1['start_point_lon'])**2)
            dframe_1=dframe_1.sort_values(by='dist_from_start')
    

            dff=dframe_1.iloc[:1]
    
            while dff.shape[0]!=sha:
                dframe_1=dframe_1.iloc[1:]
                start_point_lat=dframe_1.iloc[0]['latitude']
                start_point_lon=dframe_1.iloc[0]['longitude']
                dframe_1['dist_from_start'] = np.sqrt((dframe_1['latitude']-start_point_lat)**2+(dframe_1['longitude']-start_point_lon)**2)
                dframe_1=dframe_1.sort_values(by='dist_from_start')
        
                dff.loc[len(dff)]=dframe_1.iloc[0]
        
        
            stop = dff.iloc[0]['nom_gare']
            x=0
            y=L
            G.add_node('({}, {})'.format(stop, line), pos=(x,y), color=color_ligne, demand=0)
            for i in range(1, dff.shape[0]):
            
                stop_old = dff.iloc[i-1]['nom_gare']
                x_old=i-1
                y_old=L
            
            
                stop_new = dff.iloc[i]['nom_gare']
                x_new=i
                y_new=L
            
                dist = int(np.sqrt((x_new-x_old)**2+(y_new-y_old)**2))
            
                G.add_node("({}, {})".format(stop_new, line), pos=(x_new,y_new), color=color_ligne, demand=0)
                G.add_edge("({}, {})".format(stop_old, line),"({}, {})".format(stop_new, line), weight=dist, color=color_ligne, capacity= 10000)
                G.add_edge("({}, {})".format(stop_new, line),"({}, {})".format(stop_old, line), weight=dist, color=color_ligne, capacity= 10000)
                i = i + 1
        
            L=L+1
            
            for stop in dff['nom_gare'].to_list():
                
                for lineprime in LINES:
                    
                    if lineprime!=line and stop in dframe.loc[dframe.ligne==lineprime]['nom_gare'].to_list():
                        
                        TS_COST = int(10) 
                        G.add_edge("({}, {})".format(stop, line),"({}, {})".format(stop, lineprime), weight=TS_COST, color=[0,0,0], capacity= 10000)
                        
                    else:
                        None 
            
            
            
            
            
            
            
        

    
    else:
        
        
        for line, end in zip(LINES, START):
        
            dframe_1 = dframe.loc[dframe.ligne==line]
            sha = dframe_1.shape[0]
            color_ligne = cls.to_rgb(dframe_1.loc[dframe_1.ligne==line]['color'].unique().item())
    
    
            dframe_1['start_point_lat']=dframe_1.loc[dframe_1.nom_gare==end]['latitude'].item()
            dframe_1['start_point_lon']=dframe_1.loc[dframe_1.nom_gare==end]['longitude'].item()
            dframe_1['dist_from_start'] = np.sqrt((dframe_1['latitude']-dframe_1['start_point_lat'])**2+(dframe_1['longitude']-dframe_1['start_point_lon'])**2)
            dframe_1=dframe_1.sort_values(by='dist_from_start')
    

            dff=dframe_1.iloc[:1]
    
            while dff.shape[0]!=sha:
                dframe_1=dframe_1.iloc[1:]
                start_point_lat=dframe_1.iloc[0]['latitude']
                start_point_lon=dframe_1.iloc[0]['longitude']
                dframe_1['dist_from_start'] = np.sqrt((dframe_1['latitude']-start_point_lat)**2+(dframe_1['longitude']-start_point_lon)**2)
                dframe_1=dframe_1.sort_values(by='dist_from_start')
        
                dff.loc[len(dff)]=dframe_1.iloc[0]
        
        
            
            
            stop = dff.iloc[0]['nom_gare']
            x=dff.iloc[0]['latitude']
            y=dff.iloc[0]['longitude']
            G.add_node('({}, {})'.format(stop, line), pos=(x,y), color=color_ligne, demand=0)
            for i in range(1, dff.shape[0]):
            
                stop_old = dff.iloc[i-1]['nom_gare']
                x_old=dff.iloc[i-1]['latitude']
                y_old=dff.iloc[i-1]['longitude']
            
            
                stop_new = dff.iloc[i]['nom_gare']
                x_new=dff.iloc[i]['latitude']
                y_new=dff.iloc[i]['longitude']
            
                #dist = int(round(10000*np.sqrt((x_new-x_old)**2+(y_new-y_old)**2)))
                dist = int(round((geopy.distance.geodesic((x_old, y_old), (x_new,y_new)).m)/420))
            
                G.add_node("({}, {})".format(stop_new, line), pos=(x_new,y_new), color=color_ligne, demand=0)
                G.add_edge("({}, {})".format(stop_old, line),"({}, {})".format(stop_new, line), weight=dist, color=color_ligne, capacity= 10000)
                G.add_edge("({}, {})".format(stop_new, line),"({}, {})".format(stop_old, line), weight=dist, color=color_ligne, capacity= 10000)
        
            
            
            for stop in dff['nom_gare'].to_list():
                
                for lineprime in LINES:
                    
                    if lineprime!=line and stop in dframe.loc[dframe.ligne==lineprime]['nom_gare'].to_list():
                        
                        TS_COST = int(10) 
                        G.add_edge("({}, {})".format(stop, line),"({}, {})".format(stop, lineprime), weight=TS_COST, color=[0,0,0], capacity= 10000)
                        
                    else:
                        None
                    
            
            
            
        
    
    
    
    
    return G

In [ ]:
def draw_graph(graph, label=False):
    
    TS_edges=[(u,v) for u,v,e in graph.edges(data=True) if e['color']==[0, 0, 0]]
    S_edges=[(u,v) for u,v,e in graph.edges(data=True) if e['color']!=[0, 0, 0]]
    
    TS_COLOR= [e['color'] for u,v,e in graph.edges(data=True) if e['color']==[0, 0, 0]]
    
    S_COLOR= [e['color'] for u,v,e in graph.edges(data=True) if e['color']!=[0, 0, 0]]
    
    pos=nx.get_node_attributes(graph,'pos')
    ncolor =nx.get_node_attributes (graph,'color')
            
    plt.figure(3,figsize=(14,7)) 
    
    nx.draw(graph, pos=pos, nodelist=list(graph.nodes()), edgelist=S_edges, with_labels=label,  node_size=80, verticalalignment='top', edge_color=S_COLOR, node_color=list(ncolor.values()),  width=2, arrowsize =6)
    
    if len(TS_edges)!=0:
        
        TS = nx.draw_networkx_edges(graph, pos=pos, edgelist=TS_edges, width = 1, edge_color=TS_COLOR,  arrowsize=1)
        #S = nx.draw_networkx_edges(graph, pos=pos, edgelist=S_edges, width = 2, edge_color=S_COLOR, arrowsize=15)
        for patch in TS:
            patch.set_linestyle('dotted')
    else:
        None

    
    
    return plt.show()

In [ ]:
def all_min_paths(H, DEPART, ARRIVE):
    
    
    LINES=['1', '2', '3', '4', '5', '6', '8', '9', '11', '12', '14', 'M3bis']
    
    costs=list()
    flows=list()
    
    for startline in LINES:
        for endline in LINES:
            graph=H.copy()
            combstart=[name for name,e in graph.nodes(data=True) if name=="({}, {})".format(DEPART, startline)]
            combend=[name for name,e in graph.nodes(data=True) if name=="({}, {})".format(ARRIVE, endline)]
            if len(combstart)!=0 and len(combend)!=0:
                
                graph.nodes["({}, {})".format(DEPART, startline)]['demand']=-1
                graph.nodes["({}, {})".format(ARRIVE, endline)]['demand']=1
                
                try:
                    flowDict = nx.min_cost_flow(graph)
                    costs.append(nx.cost_of_flow(graph, flowDict))
                    flows.append(flowDict)
                except:
                    continue
                    
            else:
                continue
                
    if len(costs)==0:
        print('No path has been found')
    else:
        None
    
    
    
    return costs, flows

In [ ]:
def min_path(H, DEPART, ARRIVE):
    
    
    LINES=['1', '2', '3', '4', '5', '6', '8', '9', '11', '12', '14', 'M3bis']
    
    costs=list()
    flows=list()
    
    for startline in LINES:
        for endline in LINES:
            graph=H.copy()
            combstart=[name for name,e in graph.nodes(data=True) if name=="({}, {})".format(DEPART, startline)]
            combend=[name for name,e in graph.nodes(data=True) if name=="({}, {})".format(ARRIVE, endline)]
            if len(combstart)!=0 and len(combend)!=0:
                
                graph.nodes["({}, {})".format(DEPART, startline)]['demand']=-1
                graph.nodes["({}, {})".format(ARRIVE, endline)]['demand']=1
                
                try:
                    flowDict = nx.min_cost_flow(graph)
                    costs.append(nx.cost_of_flow(graph, flowDict))
                    flows.append(flowDict)
                except:
                    continue
                    
            else:
                continue
                
    if len(costs)==0:
        print('No path has been found, you should walk ;)')
    else:
        None
    
    bestcost = min(costs)
    bestindex = costs.index(bestcost)
    bestflow = flows[bestindex]
    
    
    
    
    return bestcost, bestflow

In [ ]:
def track_min_path(graph, flowdict):
    
    Gr = nx.DiGraph()
    pos = nx.get_node_attributes(graph, 'pos')
    col = nx.get_node_attributes(graph, 'color')
    
    for key, value in zip(flowdict.keys(),flowdict.values()):
        for k,v in value.items():
            if v!=0:
                node_key = [(name, e) for name,e in graph.nodes(data=True) if name==key]
                node_k = [(name, e) for name,e in graph.nodes(data=True) if name==k]
                
                if node_key[0][1]['pos']!=node_k[0][1]['pos']:
                    Gr.add_node(node_key[0][0], pos=node_key[0][1]['pos'], color=node_key[0][1]['color'])
                    Gr.add_node(node_k[0][0], pos=node_k[0][1]['pos'], color=node_k[0][1]['color'])
                else:
                    Gr.add_node(node_key[0][0], pos=node_key[0][1]['pos'], color=node_key[0][1]['color'])
                
                Gr.add_edge(node_key[0][0], node_k[0][0], color = node_key[0][1]['color'])
                

                
    return Gr